<div style="text-align: center; font-size: 20pt; font-weight: bold; margin: 1em 0em 1em 0em">Climate Change Hackathon: Climate data preprocessing</div>

# Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date

# Import the raw Data

In [2]:
df = pd.read_csv('Data/climate-daily.csv')

In [3]:
df.columns

Index(['x', 'y', 'STATION_NAME', 'STN_ID', 'CLIMATE_IDENTIFIER', 'ID',
       'LOCAL_DATE', 'PROVINCE_CODE', 'LOCAL_YEAR', 'LOCAL_MONTH', 'LOCAL_DAY',
       'MEAN_TEMPERATURE', 'MEAN_TEMPERATURE_FLAG', 'MIN_TEMPERATURE',
       'MIN_TEMPERATURE_FLAG', 'MAX_TEMPERATURE', 'MAX_TEMPERATURE_FLAG',
       'TOTAL_PRECIPITATION', 'TOTAL_PRECIPITATION_FLAG', 'TOTAL_RAIN',
       'TOTAL_RAIN_FLAG', 'TOTAL_SNOW', 'TOTAL_SNOW_FLAG', 'SNOW_ON_GROUND',
       'SNOW_ON_GROUND_FLAG', 'DIRECTION_MAX_GUST', 'DIRECTION_MAX_GUST_FLAG',
       'SPEED_MAX_GUST', 'SPEED_MAX_GUST_FLAG', 'COOLING_DEGREE_DAYS',
       'COOLING_DEGREE_DAYS_FLAG', 'HEATING_DEGREE_DAYS',
       'HEATING_DEGREE_DAYS_FLAG', 'MIN_REL_HUMIDITY', 'MIN_REL_HUMIDITY_FLAG',
       'MAX_REL_HUMIDITY', 'MAX_REL_HUMIDITY_FLAG'],
      dtype='object')

In [4]:
# keep only the data we want to use
df0 = df[['STATION_NAME','LOCAL_YEAR', 'LOCAL_MONTH', 'LOCAL_DAY',
    'MEAN_TEMPERATURE', 'MIN_TEMPERATURE', 'MAX_TEMPERATURE',
    'TOTAL_PRECIPITATION', 'TOTAL_RAIN', 'TOTAL_SNOW', 'SNOW_ON_GROUND',
    'DIRECTION_MAX_GUST','SPEED_MAX_GUST', 'COOLING_DEGREE_DAYS',
    'HEATING_DEGREE_DAYS','MIN_REL_HUMIDITY','MAX_REL_HUMIDITY']]

In [5]:
# extract each station's data
station_names = df0['STATION_NAME'].unique().tolist()
station_names

['MONTREAL/PIERRE ELLIOTT TRUDEAU INTL',
 'MONTREAL/ST-HUBERT',
 'MONTREAL/PIERRE ELLIOTT TRUDEAU INTL A']

# Create 3 separate DataFrame: one per station in Montreal

In [6]:
df1 = df0.loc[df0.STATION_NAME=='MONTREAL/PIERRE ELLIOTT TRUDEAU INTL']
df1 = df1[df1.columns[1:]]

In [7]:
df2 = df0.loc[df0.STATION_NAME=='MONTREAL/ST-HUBERT']
df2 = df2[df2.columns[1:]]

In [8]:
df3 = df0.loc[df0.STATION_NAME=='MONTREAL/PIERRE ELLIOTT TRUDEAU INTL A']
df3 = df3[df3.columns[1:]]

# Aggregate the data from days to weeks

In [9]:
def aggregate_data_per_week(df):
    
    # aggregate the lines per weeks
    df_per_week = []
    for year in df['LOCAL_YEAR'].unique():
        df_y = df.loc[df.LOCAL_YEAR==year]
        cpt=0
        for month in df_y['LOCAL_MONTH'].unique():
            df_y_m = df_y.loc[df_y.LOCAL_MONTH==month]
            df_y_m_s = []
            n_week_memory = None
            for index, row in df_y_m.iterrows():
                line = row.values.tolist()
                day = line[2]
                n_year, n_week = date(int(year), int(month), int(day)).isocalendar()[:2]
                id_week = int(n_year)*100+int(n_week)
                df_y_m_s.append([id_week] + line[3:])
                if n_week_memory is None:
                    n_week_memory = n_week
                elif n_week_memory<n_week:
                    df_per_week.append(df_y_m_s)
                    df_y_m_s = []
    
    # make the summary of each week
    # the columns are: 'MEAN_TEMPERATURE', 'MIN_TEMPERATURE', 'MAX_TEMPERATURE',
    #                  'TOTAL_PRECIPITATION', 'TOTAL_RAIN', 'TOTAL_SNOW', 'SNOW_ON_GROUND',
    #                  'DIRECTION_MAX_GUST','SPEED_MAX_GUST', 'COOLING_DEGREE_DAYS',
    #                  'HEATING_DEGREE_DAYS'
    
    df_per_week_summary = []
    
    for data_week in df_per_week:
        data_week = np.array(data_week)
        summary = [int(data_week[0,0])] # Id week
        # summary of the week
        summary.append(np.nanmean(data_week[:,1])) # Mean temp
        summary.append(np.nanmin(data_week[:,2])) # min temp
        summary.append(np.nanmax(data_week[:,3])) # max temp
        summary.append(np.nansum(data_week[:,4])) # total precipitation
        summary.append(np.nansum(data_week[:,5])) # total rain
        summary.append(np.nansum(data_week[:,6])) # total snow
        summary.append(np.nanmean(data_week[:,8])) # direction max gust
        summary.append(np.nanmax(data_week[:,9])) # speed max gust
        summary.append(np.nanmean(data_week[:,10])) # cooling degree days
        summary.append(np.nanmean(data_week[:,11])) # heating degree days
        summary.append(len(np.where(data_week[:,3]>20)[0])) # nb day per week where temp >20
        summary.append(len(np.where(data_week[:,3]>25)[0])) # nb day per week where temp >25
        summary.append(len(np.where(data_week[:,3]>30)[0])) # nb day per week where temp >30
        summary.append(len(np.where(data_week[:,2]<10)[0])) # nb day per week where temp <10
        summary.append(len(np.where(data_week[:,2]<0)[0])) # nb day per week where temp <0
        summary.append(len(np.where(data_week[:,2]<-5)[0])) # nb day per week where temp <-5
        summary.append(len(np.where(data_week[:,2]<-10)[0])) # nb day per week where temp <-10
        summary.append(len(np.where(data_week[:,4]>5)[0])) # nb day day per week where precipitation > 5
    
        df_per_week_summary.append(summary)
    
    df_week = pd.DataFrame(df_per_week_summary, columns=['id_week',
    'MEAN_TEMPERATURE', 'MIN_TEMPERATURE', 'MAX_TEMPERATURE',
    'TOTAL_PRECIPITATION', 'TOTAL_RAIN', 'TOTAL_SNOW',
    'DIRECTION_MAX_GUST','SPEED_MAX_GUST', 'COOLING_DEGREE_DAYS',
    'HEATING_DEGREE_DAYS',
    'nb_j_t_sup20_1w', 'nb_j_t_sup25_1w', 'nb_j_t_sup30_1w', 'nb_j_t_inf_10_1w', 'nb_j_t_inf_0_1w', 'nb_jt__inf_m5_1w',
    'nb_j_t_inf_m10_1w', 'nb_j_precip_sup_5_1w'])
    
    return(df_week)

In [10]:
df1_w = aggregate_data_per_week(df1)
df2_w = aggregate_data_per_week(df2)
df3_w = aggregate_data_per_week(df3)

L:\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: Mean of empty slice
L:\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: All-NaN slice encountered
L:\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: All-NaN slice encountered
L:\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: Mean of empty slice
L:\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: RuntimeWarning: Mean of empty slice
L:\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in greater
L:\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in greater
L:\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in greater
L:\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in less
L:\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: invalid value

# Aggregate the data from the 3 stations

In [11]:
data_week = []

for week in df1_w.id_week.unique()[:]:
    
    df1_week = df1_w.loc[df1_w.id_week==week].values
    
    # if there is data for the 3 station
    if week in df2_w.id_week.unique() and week in df3_w.id_week.unique():
        df2_week = df2_w.loc[df2_w.id_week==week].values
        df3_week = df3_w.loc[df3_w.id_week==week].values
        # concatenate the 3 lines, take the mean
        data_week.append(np.nanmean(np.concatenate((df1_week, df2_week, df3_week), axis=0), axis=0))
        
    # if there is data for only station 1 and 2
    elif week in df2_w.id_week.unique():
        df2_week = df2_w.loc[df2_w.id_week==week].values
        # take the mean of the 2 values
        data_week.append(np.nanmean(np.concatenate((df1_week, df2_week), axis=0), axis=0))
    
    # if there is data for only station 1 and 3
    elif week in df3_w.id_week.unique():
        df3_week = df3_w.loc[df3_w.id_week==week].values
        # take the mean of the 2 values
        data_week.append(np.nanmean(np.concatenate((df1_week, df3_week), axis=0), axis=0))
    
    # else there is only data for station 1
    else:
        data_week.append(df1_week[0])

# Augment the data with rolling averages/min/max etc...

In [12]:
all_data = []

# initialize the memories
week_m1 = np.reshape(data_week[0], (1, len(data_week[0])))
week_m2 = np.reshape(data_week[0], (1, len(data_week[0])))
week_m3 = np.reshape(data_week[0], (1, len(data_week[0])))

for week in data_week:
    
    liste_week = week.tolist()
    
    # data from last weeks
    week0 = np.reshape(week, (1,len(week)))
    weeks_4 = np.concatenate((week_m3, week_m2, week_m1, week0), axis=0)
    weeks_3 = np.concatenate((week_m2, week_m1, week0), axis=0)
    weeks_2 = np.concatenate((week_m1, week0), axis=0)
    
    for weeks in [weeks_4, weeks_3, weeks_2]:
    
        liste_week += [np.nanmean(weeks[:,1], axis=0)] # mean temp
        liste_week += [np.nanmin(weeks[:,2], axis=0)] # min temp
        liste_week += [np.nanmax(weeks[:,3], axis=0)] # max temp
        liste_week += [np.nansum(weeks[:,4], axis=0)] # tot precipitation
        liste_week += [np.nansum(weeks[:,5], axis=0)] # tot rain
        liste_week += [np.nansum(weeks[:,6], axis=0)] # tot snow
        liste_week += [np.nansum(weeks[:,11], axis=0)] # nb days per x weeks temp>20
        liste_week += [np.nansum(weeks[:,12], axis=0)] # nb days per x weeks temp>25
        liste_week += [np.nansum(weeks[:,13], axis=0)] # nb days per x weeks temp>30
        liste_week += [np.nansum(weeks[:,14], axis=0)] # nb days per x weeks temp<10
        liste_week += [np.nansum(weeks[:,15], axis=0)] # nb days per x weeks temp<0
        liste_week += [np.nansum(weeks[:,16], axis=0)] # nb days per x weeks temp<-5
        liste_week += [np.nansum(weeks[:,17], axis=0)] # nb days per x weeks temp<-10
        liste_week += [np.nansum(weeks[:,18], axis=0)] # nb days per x weeks precipitation > 5
    
    week_m3 = week_m2
    week_m2 = week_m1
    week_m1 = week0
    
    all_data.append(liste_week)

In [13]:
Columns = ['id_week', 'MEAN_TEMPERATURE', 'MIN_TEMPERATURE', 'MAX_TEMPERATURE', 'TOTAL_PRECIPITATION',
           'TOTAL_RAIN', 'TOTAL_SNOW', 'DIRECTION_MAX_GUST','SPEED_MAX_GUST', 'COOLING_DEGREE_DAYS',
           'HEATING_DEGREE_DAYS', 'nb_j_t_sup20_1w', 'nb_j_t_sup25_1w', 'nb_j_t_sup30_1w', 'nb_j_t_inf_10_1w',
           'nb_j_t_inf_0_1w', 'nb_jt__inf_m5_1w', 'nb_j_t_inf_m10_1w', 'nb_j_precip_sup_5_1w',
          'mean_t_4w', 'min_t_4w', 'max_t_4w', 'tot_precip_4w', 'tot_rain_4w', 'tot_snow_4w',
          'nb_j_t_sup20_4w', 'nb_j_t_sup25_4w', 'nb_j_t_sup30_4w', 'nb_j_t_inf_10_4w',
           'nb_j_t_inf_0_4w', 'nb_jt__inf_m5_4w', 'nb_j_t_inf_m10_4w', 'nb_j_precip_sup_5_4w',
          'mean_t_3w', 'min_t_3w', 'max_t_3w', 'tot_precip_3w', 'tot_rain_3w', 'tot_snow_3w',
          'nb_j_t_sup20_3w', 'nb_j_t_sup25_3w', 'nb_j_t_sup30_3w', 'nb_j_t_inf_10_3w',
           'nb_j_t_inf_0_3w', 'nb_jt__inf_m5_3w', 'nb_j_t_inf_m10_3w', 'nb_j_precip_sup_5_3w',
          'mean_t_2w', 'min_t_4w', 'max_t_2w', 'tot_precip_2w', 'tot_rain_2w', 'tot_snow_2w',
          'nb_j_t_sup20_2w', 'nb_j_t_sup25_2w', 'nb_j_t_sup30_2w', 'nb_j_t_inf_10_2w',
           'nb_j_t_inf_0_2w', 'nb_jt__inf_m5_2w', 'nb_j_t_inf_m10_2w', 'nb_j_precip_sup_5_2w']

In [14]:
df_final = pd.DataFrame(all_data[4:], columns=Columns)
df_final.fillna(method='ffill', inplace=True)
df_final.to_csv('climate_per_week_final.csv')